# French to English translation system using the **LSTM - Long Short-Term Memory** algorithm
## Dataset available on KAgle: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench?resource=download

In [2]:

# Importer les bibliothèques
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# Charger les modèles de langage français et anglais
modele_francais = spacy.load('fr_core_news_sm')
modele_anglais = spacy.load('en_core_web_sm')

In [ ]:

# load dataa
data = pd.read_csv("english_french.csv", delimiter=",", header=None, names=["English", "French"])

# adjust it, remove index
data = data[["English", "French"]].reset_index(drop=True)


data = data.head(5000) # use all dataset for better resultssss


In [33]:
data.shape

(5000, 2)

In [34]:
print(data.head())
print(data.columns)

                   English                  French
0  English words/sentences  French words/sentences
1                      Hi.                  Salut!
2                     Run!                 Cours !
3                     Run!                Courez !
4                     Who?                   Qui ?
Index(['English', 'French'], dtype='object')


In [35]:
# Fusionner et tokeniser la data
data["tokens_fr"] = data.iloc[:, 1].apply(modele_francais)
data["tokens_en"] = data.iloc[:, 0].apply(modele_anglais)

In [36]:
# Obtenir le vocabulaire unique pour chaque langue
vocabulaire_fr = set([token.text for doc in data["tokens_fr"] for token in doc])
vocabulaire_en = set([token.text for doc in data["tokens_en"] for token in doc])

In [37]:
# Créer des dictionnaires de tokens vers index pour chaque vocabulaire
index_fr = {token: i + 1 for i, token in enumerate(vocabulaire_fr)}
index_en = {token: i + 1 for i, token in enumerate(vocabulaire_en)}

In [38]:
# Fonctions pour convertir des séquences de tokens en indices
def convertir_tokens_fr(tokens):
    return [index_fr.get(token.text, 0) for token in tokens]

def convertir_tokens_en(tokens):
    return [index_en.get(token.text, 0) for token in tokens]

In [39]:
# Appliquer la conversion des tokens en indices
data["indices_fr"] = data["tokens_fr"].apply(convertir_tokens_fr)
data["indices_en"] = data["tokens_en"].apply(convertir_tokens_en)

In [40]:
# Utiliser Keras pour uniformiser la longueur des séquences
sequence_fr = tf.keras.preprocessing.sequence.pad_sequences(data["indices_fr"], padding="post")
sequence_en = tf.keras.preprocessing.sequence.pad_sequences(data["indices_en"], padding="post")

In [41]:
# Taille du vocabulaire
taille_vocabulaire_fr = len(index_fr)
taille_vocabulaire_en = len(index_en)

# Définir le modèle
modele_seq2seq = tf.keras.Sequential([
    # Couche d'Embedding pour les mots
    tf.keras.layers.Embedding(input_dim=taille_vocabulaire_fr + 1, output_dim=128, mask_zero=True),

    # Couche LSTM bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),

    # Deuxième couche LSTM bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=False)),

    # Répéter le vecteur pour correspondre aux séquences de sortie
    tf.keras.layers.RepeatVector(sequence_en.shape[1]),

    # Couche LSTM supplémentaire pour le décodeur
    tf.keras.layers.LSTM(64, return_sequences=True),

    # Couche de sortie avec Softmax pour la classification des tokens
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(taille_vocabulaire_en + 1, activation="softmax"))
])

# Compiler le modèle
modele_seq2seq.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
historique = modele_seq2seq.fit(sequence_fr, sequence_en, epochs=10, batch_size=32, validation_split=0.2)

# Évaluation du modèle sur les données d'entraînement
evaluation = modele_seq2seq.evaluate(sequence_fr, sequence_en)
print(f"Précision d'évaluation : {evaluation[1] * 100:.2f}%")


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 134ms/step - accuracy: 0.3529 - loss: 5.3561 - val_accuracy: 0.3260 - val_loss: 4.0923
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 139ms/step - accuracy: 0.3697 - loss: 3.4502 - val_accuracy: 0.3303 - val_loss: 3.8956
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 125ms/step - accuracy: 0.4273 - loss: 3.2240 - val_accuracy: 0.3358 - val_loss: 3.7613
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - accuracy: 0.4326 - loss: 3.0464 - val_accuracy: 0.3473 - val_loss: 3.6221
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.4936 - loss: 2.8663 - val_accuracy: 0.3955 - val_loss: 3.5108
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 129ms/step - accuracy: 0.5551 - loss: 2.7125 - val_accuracy: 0.4102 - val_loss: 3.4151
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 0.5692 - loss: 2.5917 - val_accuracy: 0.4205 - val_loss: 3.3014
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 145ms/step - accuracy: 0.5741 - loss: 2

In [42]:
modele_seq2seq.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (32, 13, 128)               │         346,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (32, 13, 256)               │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (32, 256)                   │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (32, 6, 256)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (32, 6, 64)                 │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (32, 6, 1388)               │          90,220 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,528,134 (13.46 MB)

 Trainable params: 1,176,044 (4.49 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,352,090 (8.97 MB)

In [43]:
#model.save("translation_model.h5")
modele_seq2seq.save('translation_model.keras')

In [ ]:
# Tester une prédiction avec des données
def tester_traduction(entree_francais):
    # Tokeniser et encoder la phrase d'entrée
    tokens = modele_francais(entree_francais)
    sequence = convertir_tokens_fr(tokens)
    sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=sequence_fr.shape[1], padding="post")

    # Générer la prédiction
    prediction = modele_seq2seq.predict(sequence)
    prediction_indices = np.argmax(prediction[0], axis=-1)

    # Décoder la séquence prédite en mots anglais
    mots_predits = [list(index_en.keys())[list(index_en.values()).index(i)] for i in prediction_indices if i in index_en.values()]
    traduction = " ".join(mots_predits)

    print(f"Traduction prédite : {traduction}")

# Exemple de test de traduction
tester_traduction("Bonjour, comment allez-vous ?")
